In [1]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
milapgohil_car_dataset_path = kagglehub.dataset_download('milapgohil/car-dataset')

print('Data source import complete.')


Data source import complete.


In [2]:
https://www.kaggle.com/code/alperenaslann/car-price-prediction-with-multiple-models?scriptVersionId=247292389&cellId=2

SyntaxError: invalid syntax (ipython-input-2-2369834404.py, line 1)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, ExtraTreesRegressor
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRFRegressor
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df.isnull().sum()

In [ ]:
df.info()


In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(x='Fuel Type', data=df, palette='viridis')
plt.title('Count of Cars by Fuel Type')
plt.xlabel('Fuel Type')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.show()

plt.figure(figsize=(10, 6))
sns.countplot(x='Transmission', data=df, palette='viridis')
plt.title('Count of Cars by Transmission Type')
plt.xlabel('Transmission Type')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.show()

plt.figure(figsize=(10, 6))
sns.scatterplot(x='Kilometers', y='Price', data=df, hue='Fuel Type', palette='viridis', alpha=0.7)
plt.title('Price vs Kilometers by Fuel Type')
plt.grid(True, linestyle='--', alpha=0.5)
plt.xlabel('Kilometers')
plt.ylabel('Price')
plt.legend(title='Fuel Type')
plt.show()

plt.figure(figsize=(12, 6))
sns.boxplot(x='Year', y='Price', data=df, hue='Transmission', palette='viridis')
plt.title('Price vs Year by Transmission Type')
plt.xlabel('Year')
plt.ylabel('Price')
plt.legend(title='Transmission Type', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xticks(rotation=45)
plt.show()

plt.figure(figsize=(10, 6))
sns.histplot(df['Price'], bins=30, kde=True, color='blue')
plt.title('Distribution of Car Prices')
plt.xlabel('Price')
plt.ylabel('Frequency')
plt.show()

In [ ]:
X = df.drop(['Price'], axis=1)
y = df['Price']

le = LabelEncoder()
X = X.apply(lambda col: le.fit_transform(col) if col.dtype == 'object' else col)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
df.head()

In [ ]:
df.info()


In [ ]:
df = pd.get_dummies(df, drop_first=True)

In [ ]:
df.head()

In [ ]:
models = {
    'Linear Regression': LinearRegression(n_jobs=-1),
    'KNeighbors': KNeighborsRegressor(n_jobs=-1),
    'LightGBM': LGBMRegressor(n_jobs=-1),
    'XGBRF': XGBRFRegressor(n_jobs=-1),
    'XGBoost': XGBRegressor(n_jobs=-1),
    'Random Forest': RandomForestRegressor(n_jobs=-1),
    'Extra Trees': ExtraTreesRegressor(n_jobs=-1),
    'MLP Regressor': MLPRegressor(max_iter=1000, random_state=42),
    'HistGradientBoosting': HistGradientBoostingRegressor(),
    'Ridge Regression': Ridge(),
    'Lasso Regression': Lasso(),
    'ElasticNet Regression': ElasticNet(),
    'Decision Tree': DecisionTreeRegressor(),
    'Gradient Boosting': GradientBoostingRegressor(),
    'AdaBoost': AdaBoostRegressor(),
}

results = {}
for name, model in models.items():
    try:
        m = model.fit(X_train, y_train)
        y_pred = m.predict(X_test)
        r2 = r2_score(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        mae = np.mean(np.abs(y_test - y_pred))
        mse = mean_squared_error(y_test, y_pred)
        results[name] = {
            'R2 Score': r2,
            'RMSE': rmse,
            'MAE': mae,
            'MSE': mse,
        }
        print(f"{name} trained")
    except Exception as e:
        print(f"Error with {name}: {e}")

In [ ]:
results_df = pd.DataFrame(results).T
results_df = results_df.sort_values(by='R2 Score', ascending=False)
print(results_df)

plt.figure(figsize=(12, 8))
sns.barplot(x=results_df.index, y='R2 Score', data=results_df, palette='viridis')
plt.title('Model Performance Comparison')
plt.xlabel('Model')
plt.ylabel('R2 Score')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
best_model_name = results_df['R2 Score'].idxmax()
best_model = models[best_model_name]
print(f"The best model is: {best_model_name} with R2 Score: {results_df['R2 Score'].max()}")

best_model.fit(X_train, y_train)
y_pred_best = best_model.predict(X_test)

actual_vs_predicted = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred_best})
print(actual_vs_predicted.head())
plt.figure(figsize=(10, 6))
sns.scatterplot(x='Actual', y='Predicted', data=actual_vs_predicted, alpha=0.7)
plt.plot([actual_vs_predicted['Actual'].min(), actual_vs_predicted['Actual'].max()],
         [actual_vs_predicted['Actual'].min(), actual_vs_predicted['Actual'].max()],
         color='red', linestyle='--')
plt.title('Actual vs Predicted Prices')
plt.xlabel('Actual Prices')
plt.ylabel('Predicted Prices')
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()